<a href="https://colab.research.google.com/github/MichelePresti/NAS_MachineLearningDeepLearning/blob/main/PersonalNASWOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchviz

In [ ]:
import torch
import torchvision
import numpy as np
from torchviz import make_dot

""" TENSOR CREATION """

# data = [[1, 2], [3, 4]]
# x_data = torch.tensor(data)

# np_array = np.array(data)
# x_np = torch.from_numpy(np_array)

# x_ones = torch.ones_like(x_data)
# # print(f"Ones Tensor: \n {x_ones} \n")

# x_rand = torch.rand_like(x_data, dtype=torch.float)
# # print(f"Random Tensor: \n {x_rand} \n")

# shape = (2, 3,)
# rand_tensor = torch.rand(shape)
# ones_tensor = torch.ones(shape)
# zeros_tensor = torch.zeros(shape)

# # print(f"Random Tensor: \n {rand_tensor} \n")
# # print(f"Ones Tensor: \n {ones_tensor} \n")
# # print(f"Zeros Tensor: \n {zeros_tensor}")

# tensor = torch.rand(3, 4)

# # print(f"Shape of tensor: {tensor.shape}")
# # print(f"Datatype of tensor: {tensor.dtype}")
# # print(f"Device tensor is stored on: {tensor.device}")

# # We move our tensor to the GPU if available
# if torch.cuda.is_available():
#     tensor = tensor.to('cuda')
#     # print(f"Device tensor is stored on: {tensor.device}")

# # Tensor Slicing
# tensor = torch.ones(4, 4)
# tensor[:, 1] = 0
# # print(tensor)

# # Join Tensor
# t1 = torch.cat([tensor, tensor, tensor], dim=1)
# # print(t1)

# # This computes the element-wise product
# # print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
# # Alternative syntax:
# # print(f"tensor * tensor \n {tensor * tensor}")

# # print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
# # Alternative syntax:
# # print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

# # In-place operations Operations that have a _ suffix are in-place. For example: x.copy_(y), x.t_(), will change x.
# # print(tensor, "\n")
# tensor.add_(5)
# # print(tensor)

# A GENTLE INTRODUCTION TO TORCH.AUTOGRAD

# For this example, we load a pretrained resnet18 model from torchvision.
# We create a random data tensor to represent a single image with 3 channels, and height & width of 64,
# and its corresponding label initialized to some random values. Label in pretrained models has shape (1,1000)
# model = torchvision.models.resnet18(pretrained=True)
# data = torch.rand(1, 3, 64, 64)
# labels = torch.rand(1, 1000)

# # Next, we run the input data through the model through each of its layers to make a prediction. This is the forward pass.
# prediction = model(data)

# # We use the model’s prediction and the corresponding label to calculate the error (loss).
# # The next step is to backpropagate this error through the network.
# # Backward propagation is kicked off when we call .backward() on the error tensor.
# # Autograd then calculates and stores the gradients for each model parameter in the parameter’s .grad attribute.
# loss = (prediction - labels).sum()
# loss.backward()

# # Next, we load an optimizer, in this case SGD with a learning rate of 0.01 and momentum of 0.9. We register all the parameters of the model in the optimizer.
# optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

# # Finally, we call .step() to initiate gradient descent. The optimizer adjusts each parameter by its gradient stored in .grad.
# optim.step()

# make_dot(model(data)) # .render("rnn_torchviz", format="png")

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# net = Net()
# print(net)

# params = list(net.parameters())

# input = torch.randn(1, 1, 32, 32)
# out = net(input)
# # make_dot(out)
# # print(out)

# net.zero_grad()
# out.backward(torch.randn(1, 10))

# output = net(input)
# target = torch.randn(10)  # a dummy target, for example
# target = target.view(1, -1)  # make it the same shape as output
# criterion = nn.MSELoss()

# loss = criterion(output, target)
# print(loss)

# print(loss.grad_fn)  # MSELoss
# print(loss.grad_fn.next_functions[0][0])  # Linear
# print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# net.zero_grad()     # zeroes the gradient buffers of all parameters

# print('conv1.bias.grad before backward')
# print(net.conv1.bias.grad)

# loss.backward()

# print('conv1.bias.grad after backward')
# print(net.conv1.bias.grad)

# import torch.optim as optim

# # create your optimizer
# optimizer = optim.SGD(net.parameters(), lr=0.01)

# # in your training loop:
# optimizer.zero_grad()   # zero the gradient buffers
# output = net(input)
# loss = criterion(output, target)
# loss.backward()
# optimizer.step()    # Does the update



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import timeit

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

for name, module in net.named_modules():
    print(name, module)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = timeit.default_timer()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

stop = timeit.default_timer()
exec_time = stop - start;
print('Finished Training')
print(f'Execution Time: [{exec_time}]')


# Save Result
# PATH = './cifar_net.pth'
# torch.save(net.state_dict(), PATH)

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

print(net.named_modules())

In [ ]:
!cp -r /content/drive/MyDrive/nas-wot-lib/NAS_MachineLearningDeepLearning/main.py .

In [ ]:
!pip install nas-bench-201

In [ ]:
# NAS WOT Algorithm

import numpy as np
import torch
import random
import argparse
from searchspace.searchspace import NasBench201
from datasets import datasets as data
from score.score import ScoreAgent as Score

# PARSING ARGUMENT

# parser = argparse.ArgumentParser(description='NAS Without Training')
# parser.add_argument('--data_loc', default='../cifardata/', type=str, help='dataset folder')
# parser.add_argument('--api_loc', default='../NAS-Bench-201-v1_0-e61699.pth',
#                     type=str, help='path to API')
# parser.add_argument('--save_loc', default='results', type=str, help='folder to save results')
# parser.add_argument('--save_string', default='naswot', type=str, help='prefix of results file')
# parser.add_argument('--score', default='hook_logdet', type=str, help='the score to evaluate')
# parser.add_argument('--nasspace', default='nasbench201', type=str, help='the nas search space to use')
# parser.add_argument('--batch_size', default=128, type=int)
# parser.add_argument('--repeat', default=1, type=int, help='how often to repeat a single image with a batch')
# parser.add_argument('--augtype', default='none', type=str, help='which perturbations to use')
# parser.add_argument('--sigma', default=0.05, type=float, help='noise level if augtype is "gaussnoise"')
# parser.add_argument('--GPU', default='0', type=str)
# parser.add_argument('--seed', default=1, type=int)
# parser.add_argument('--init', default='', type=str)
# parser.add_argument('--trainval', action='store_true')
# parser.add_argument('--dropout', action='store_true')
# parser.add_argument('--dataset', default='cifar10', type=str)
# parser.add_argument('--maxofn', default=1, type=int, help='score is the max of this many evaluations of the network')
# parser.add_argument('--n_samples', default=100, type=int)
# parser.add_argument('--n_runs', default=500, type=int)
# parser.add_argument('--stem_out_channels', default=16, type=int,
#                     help='output channels of stem convolution (nasbench101)')
# parser.add_argument('--num_stacks', default=3, type=int, help='#stacks of modules (nasbench101)')
# parser.add_argument('--num_modules_per_stack', default=3, type=int, help='#modules per stack (nasbench101)')
# parser.add_argument('--num_labels', default=1, type=int, help='#classes (nasbench101)')

# args = parser.parse_args()

#######################

# LOADING SEARCH SPACE ########
path = '/content/drive/MyDrive/NAS-Bench-201-v1_1-096897.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = 'cifar10'

searchspace = NasBench201(path)

################################








In [ ]:
# LOADING DATASET ##############

""" Modules for data loading can be found in datasets package"""

train_dt = data.load_dataset(dataset)
print(train_dt)

################################

In [ ]:
# SCORE INITIALIZING ##############

score = Score(searchspace)
print(score)

################################

In [ ]:
# SCORING ARCHITECTURE #########

for i, (uid, network) in enumerate(searchspace):
    try:

        # Add Hook to modules in the current network
        if 'hook_' in args.score:

            def counting_forward_hook(module_hook, inp, out):
                try:
                    if not module_hook.visited_backwards:
                        return
                    if isinstance(inp, tuple):
                        inp = inp[0]
                    inp = inp.view(inp.size(0), -1)
                    x = (inp > 0).float()
                    K = x @ x.t()
                    K2 = (1. - x) @ (1. - x.t())
                    network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                except Exception as exception:
                    print(exception)
                    pass

            def counting_backward_hook(module_hook, inp, out):
                module_hook.visited_backwards = True

            for name, module in network.named_modules():
                if 'ReLU' in str(type(module)):
                    module.register_forward_hook(counting_forward_hook)
                    module.register_backward_hook(counting_backward_hook)

        # Starting score algorithm
        network = network.to(device)
        random.seed(args.seed)
        np.random.seed(args.seed)
        torch.manual_seed(args.seed)
        s = []
        for j in range(args.maxofn):
            data_iterator = iter(train_dt)
            x, target = next(data_iterator)
            x2 = torch.clone(x)
            x2 = x2.to(device)
            x, target = x.to(device), target.to(device)
            print(x, target)
            jacobs, labels, y, out = score.get_jacobian(network, x, target, device, args)
            if 'hook_' in args.score:
                network(x2.to(device))
                s.append(score.get_score_func(args.score)(network.K, target))
            else:
                pass
                s.append(score.get_score_func(args.score)(jacobs, labels))

        score.register_score(s, i)
        """ Check other score in original file and add """

    except Exception as e:
        print(e)
        score.register_score([np.NaN], i)

################################

# SAVING RESULT ################

""" TO IMPLEMENT """

################################


In [ ]:
import numpy as np

a = [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 1, 11, 1, 1,1 ,1, 1, 1]